In [1]:
import pandas as pd
import miceforest as mf
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

General function for cleaning the data belonging to (optimistically) *any* aquifer dataset:

In [2]:
def clean_aquifers(df, X, y):
    
    # set date as the index
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df = df.set_index('Date', drop=True)
    df.index.name = None
    df.index.freq = 'D'
    
    # find the latest date of first non-null
    # amongst our targets; throw away all data
    # prior to this point in time
    sent = None
    for c in y:
        data = df[~df[c].isnull()]
        if data.shape[0] > 0:
            d = data.iloc[0].name
            if sent is None or d > sent:
                sent = d
    if sent is not None:
        df = df.loc[sent:]

    # similarly, find the earliest date of last non-null
    # among the targets, again for trimming purposes
    sent = None
    for c in y:
        data = df[~df[c].isnull()]
        if data.shape[0] > 0:
            d = data.iloc[-1].name
            if sent is None or d < sent:
                sent = d
    if sent is not None:
        df = df.loc[:sent]
        
    # 0 in the target always represents a missing value
    for c in y:
        df[c] = df[c].replace(0, float('nan'))

    # discard features with >70% nulls
    xnulls = df[X].isnull().sum() / df.shape[0]
    Xs = df[xnulls[xnulls < 0.7].index]
    X = Xs.columns
    df = Xs.join(df[y])
    
    # take the absolute value of some columns
    abs_starts = [
        'Rainfall',
        'Depth_to_Groundwater',
        'Volume'
    ]
    for beg in abs_starts:
        cols = [c for c in df.columns if c.startswith(beg)]
        for c in cols:
            df[c] = df[c].abs()

    # multiple imputation
    kernel = mf.MultipleImputedKernel(
        data=df,
        save_all_iterations=False,
        random_state=143
    )
    kernel.mice(3, verbose=False)
    df = kernel.impute_new_data(df).complete_data(0)
    
    return df

Convenience class for loading (+ cleaning) the individual datasets:

In [3]:
class AquiferDataset:
    
    def __init__(self, path, X, y):
        print(f"loading {path}")
        
        self.ycard = len(y)
        self.df = pd.read_csv(path)
        self.df = clean_aquifers(self.df, X, y)
        self.X = self.df.columns[:-self.ycard]
        self.y = self.df.columns[-self.ycard:]
        
        # center & scale the data (*fuller explanation below)
        self.scaler = StandardScaler()
        self.df = pd.DataFrame(
            self.scaler.fit_transform(self.df),
            columns=self.df.columns,
            index=self.df.index
        )
        self.mu = self.scaler.mean_
        self.sigma = self.scaler.scale_
    
    def unscale(self, v):
        if len(v.shape) == 1:
            if self.ycard != 1:
                raise ValueError()
            else:
                return v * self.sigma[0] + self.mu[0]
        elif v.shape[1] != self.ycard:
            raise ValueError()
        else:
            for i, c in enumerate(v.columns):
                v[c] = v[c] * self.sigma[i] + self.mu[i]
            return v

\* We scale the data separately because of the following conundrum: When producing predictions, we want everything z-scored so that different datasets can be compared on the same scale; but when producing predictions, we *don't* want everything z-scored, because we're interested in actual units of measurement. We have to be able to do everything under the hood with z-scores, but still produce meaningful predictions

Load each dataset and split by independent/dependent variables:

In [4]:
auser = AquiferDataset(
    '../data/raw/aquifer/auser.csv', [
        'Rainfall_Gallicano', 'Rainfall_Pontetetto', 'Rainfall_Monte_Serra',
        'Rainfall_Orentano', 'Rainfall_Borgo_a_Mozzano', 'Rainfall_Piaggione',
        'Rainfall_Calavorno', 'Rainfall_Croce_Arcana', 'Rainfall_Tereglio_Coreglia_Antelminelli',
        'Rainfall_Fabbriche_di_Vallico', 'Depth_to_Groundwater_PAG', 'Depth_to_Groundwater_DIEC',
        'Temperature_Orentano', 'Temperature_Monte_Serra', 'Temperature_Ponte_a_Moriano',
        'Temperature_Lucca_Orto_Botanico', 'Volume_POL', 'Volume_CC1', 'Volume_CC2',
        'Volume_CSA', 'Volume_CSAL', 'Hydrometry_Monte_S_Quirico', 'Hydrometry_Piaggione'
    ], [
        'Depth_to_Groundwater_SAL', 'Depth_to_Groundwater_CoS', 'Depth_to_Groundwater_LT2'
    ])

doganella = AquiferDataset(
    '../data/raw/aquifer/doganella.csv', [
        'Rainfall_Monteporzio', 'Rainfall_Velletri', 'Volume_Pozzo_1',
        'Volume_Pozzo_2', 'Volume_Pozzo_3', 'Volume_Pozzo_4', 'Volume_Pozzo_5+6',
        'Volume_Pozzo_7', 'Volume_Pozzo_8', 'Volume_Pozzo_9', 'Temperature_Monteporzio',
        'Temperature_Velletri'
    ], [
        'Depth_to_Groundwater_Pozzo_1', 'Depth_to_Groundwater_Pozzo_2',
        'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
        'Depth_to_Groundwater_Pozzo_5', 'Depth_to_Groundwater_Pozzo_6',
        'Depth_to_Groundwater_Pozzo_7', 'Depth_to_Groundwater_Pozzo_8',
        'Depth_to_Groundwater_Pozzo_9'
    ])

luco = AquiferDataset(
    '../data/raw/aquifer/luco.csv', [
        'Rainfall_Simignano', 'Rainfall_Siena_Poggio_al_Vento', 'Rainfall_Mensano',
        'Rainfall_Montalcinello', 'Rainfall_Monticiano_la_Pineta', 'Rainfall_Sovicille',
        'Rainfall_Ponte_Orgia', 'Rainfall_Scorgiano', 'Rainfall_Pentolina',
        'Rainfall_Monteroni_Arbia_Biena', 'Depth_to_Groundwater_Pozzo_1',
        'Depth_to_Groundwater_Pozzo_3', 'Depth_to_Groundwater_Pozzo_4',
        'Temperature_Siena_Poggio_al_Vento', 'Temperature_Mensano', 'Temperature_Pentolina',
        'Temperature_Monteroni_Arbia_Biena', 'Volume_Pozzo_1', 'Volume_Pozzo_3',
        'Volume_Pozzo_4'
    ], [ 'Depth_to_Groundwater_Podere_Casetta' ])

petrignano = AquiferDataset(
    '../data/raw/aquifer/petrignano.csv', [
        'Rainfall_Bastia_Umbra', 'Temperature_Bastia_Umbra',
        'Temperature_Petrignano', 'Volume_C10_Petrignano',
        'Hydrometry_Fiume_Chiascio_Petrignano'
    ], [
        'Depth_to_Groundwater_P24', 'Depth_to_Groundwater_P25'
    ])

loading ../data/raw/aquifer/auser.csv
loading ../data/raw/aquifer/doganella.csv
loading ../data/raw/aquifer/luco.csv
loading ../data/raw/aquifer/petrignano.csv


This notebook is more about the general pipeline than the individual model; our dummy model, then, will always predict the number 0.

In [5]:
class AlwaysZeroPredictor:
    
    def fit(self, X, y):
        self._outshape = 1 if len(y.shape) == 1 else y.shape[1]
        self._onedim = self._outshape == 1
        return self
    
    def predict(self, n):
        return np.zeros((n, self._outshape))

In [6]:
class TimeSeriesValidation:
    
    def __init__(self, d):
        self.log = []
        self.d = d
    
    def split(self, X, y):
        self.k = math.floor(X.shape[0] / self.d - 1)
        self.tscv = TimeSeriesSplit(self.k)
        self.X = X
        self.y = y
        return self
    
    def validate(self, m):
        for tr, tt in self.tscv.split(self.X, self.y):
            X_train, X_test = self.X.iloc[tr], self.X.iloc[tt]
            y_train, y_test = self.y.iloc[tr], self.y.iloc[tt]
            m.fit(X_train, y_train)
            y_pred = pd.DataFrame(
                m.predict(y_test.shape[0]),
                columns = y_test.columns,
                index = y_test.index
            )
            rmse = mean_squared_error(y_test, y_pred, squared=False, multioutput='raw_values')
            mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
            self.log.append({
                'train': tr,
                'test': tt,
                'model': m,
                'predictions': y_pred,
                'rmse': rmse,
                'mae': mae
            })
        return self
    
    def report(self):
        rmse = [x['rmse'] for x in self.log]
        rmse = {c: np.mean([x[i] for x in rmse]) for i, c in enumerate(self.y.columns)}
        mae = [x['mae'] for x in self.log]
        mae = {c: np.mean([x[i] for x in mae]) for i, c in enumerate(self.y.columns)}
        rmse['mean'] = np.mean(list(rmse.values()))
        mae['mean'] = np.mean(list(mae.values()))
        rmse['metric'] = 'rmse'
        mae['metric'] = 'mae'
        return rmse, mae
    
    def plot(self, body):
        rmses, maes = self.report()
        fig, axs = plt.subplots(
            len(self.y.columns),
            1,
            figsize=(20, 5 * len(self.y.columns))
        )
        if len(self.y.columns) == 1:
            axs = [axs]
        fig.suptitle(f"{body} Aquifer Predictions | RMSE {rmses['mean']} ... MAE {maes['mean']}", fontweight='bold')
        preds = [x['predictions'] for x in self.log]
        for i, c in enumerate(self.y.columns):
            axs[i].scatter(self.y.index, self.y[c], s=1, alpha=0.75)
            axs[i].set_title(f"{c} | RMSE {rmses[c]} ... MAE {maes[c]}")
            for p in preds:
                axs[i].scatter(p.index, p[c].values, s=1, c='C3', alpha=0.25)
        plt.show()

In [7]:
# this interval
# this model
# these datasets

class HeterogeneousValidation:
    
    def __init__(self, d, m, *ads):
        
        self.tsvs = []
        for ad in ads:
            tsv = TimeSeriesValidation(d)
            tsv.split(ad.df[ad.X], ad.df[ad.y])
            tsv.validate(m)
            self.tsvs.append(tsv)
    
    def verbose(self):
        pass
    
    def concise(self):
        rmses = [tsv.report()[0]['mean'] for tsv in self.tsvs]
        maes = [tsv.report()[1]['mean'] for tsv in self.tsvs]
        return np.mean(rmses), np.mean(maes)

In [9]:
hv = HeterogeneousValidation(
    7,
    AlwaysZeroPredictor(),
    auser, doganella, luco, petrignano
)

# LET IT HERE BE KNOWN

When using the method of "always predict 0" and intervals of 7 days, our scores are such:

In [10]:
hv.concise() # RMSE, MAE

(0.8350535491577347, 0.8258548852105945)